<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/RNN_CV19Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-sparse 0.6.2
    Uninstalling torch-sparse-0.6.2:
      Successfully uninstalled torch-sparse-0.6.2
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
  Found existing installation: torch-cluster 1.5.4
    Uninstalling torch-cluster-1.5.4:
      Successfully uninstalled torch-cluster-1.5.4


!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os

import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, InMemoryDataset

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on '+ str(device))

running on cuda


In [3]:
from google.colab import drive
import os.path
from os import path
import sys
from os import listdir
from os.path import isfile, join

drive.mount('/content/drive')
PATH = '/content/drive/My Drive/covid'
sys.path.append(PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class CovidDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(CovidDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+'/cleaned'
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+'/cleaned'
        
    @property
    def processed_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+PROCESSED_DIR
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+PROCESSED_DIR

    @property
    def raw_file_names(self):
      mypath = self.raw_dir
      filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
      return filenames

    @property
    def processed_file_names(self):
        return ['processed.dt']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []
        output_sequence_len = configuration['output_sequence_len']
        input_sequence_len = configuration['input_sequence_len']
        feature_length = configuration['feature_len']
        yhat_size = configuration['yhat_size']

        for raw_path in self.raw_paths:
          df = pd.read_csv(raw_path)
          for synthetic_seq in df['synthesis_seq'].unique():
            synthetic_data = df[df['synthesis_seq']==synthetic_seq]

            for country in synthetic_data['countryterritoryCode'].unique():
              country_data = synthetic_data[synthetic_data['countryterritoryCode'] == country]
              popData2018 = country_data['popData2018'].values[0]
              _country_code = country_data['_country_code'].values[0]
              
              country_data_i = country_data[:-output_sequence_len]
              country_data_o = country_data[input_sequence_len:]
              
              country_data_array = np.array([country_data_i['cases'].to_numpy(),
                                             country_data_i['deaths'].to_numpy(),
                                             country_data_i['popData2018'].to_numpy(),
                                             country_data_i['_country_code'].to_numpy(),
                                             country_data_i['countriesAndTerritories'].to_numpy(),
                                             country_data_i['geoId'].to_numpy(),
                                             country_data_i['countryterritoryCode'].to_numpy(),
                                             country_data_i['continentExp'].to_numpy()
                                             ])
              feature_length = len(country_data_array)
              country_data_array = country_data_array.reshape(feature_length,len(country_data_i))

              country_data_array_y = np.array([country_data_o['cases'].to_numpy(), country_data_o['deaths'].to_numpy()])
              country_data_array_y = country_data_array_y.reshape(2,len(country_data_o))

              x = country_data_array[:feature_length].T
              y = country_data_array_y[:yhat_size].T

              sets =0
              x_list = []
              dict_x = dict()
              for i in range(input_sequence_len):
                array_len = ((len(x) -i) - ((len(x)-i)%input_sequence_len))+i
                if array_len <= 0:
                  continue
                sets = int( array_len/ input_sequence_len)
                if sets <= 0:
                  continue
                #print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
                x_temp = x[i:array_len].T.reshape(sets,feature_length,input_sequence_len)
                x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
                uniq_keys = np.array([i+(input_sequence_len*k) for k in range(input_sequence_len)])
                
                arrays_split = np.hsplit(x_temp,sets)
                dict_x.update(dict(zip(uniq_keys, arrays_split)))
              
              dict_y = dict()
              y_list = []
              for i in range(output_sequence_len):
                array_len_y = (len(y)-i) - ((len(y)  - i)%output_sequence_len)+i
                if array_len_y <= 0:
                  continue
                sets = int(array_len_y / output_sequence_len)
                if sets <= 0:
                  continue
                
                #print('output seq : ', i , ' ', array_len_y , ' ',array_len_y-(i) , ' number of sets : ', sets)
                y_temp = y[i:array_len_y].T.reshape(sets, 2, output_sequence_len)
                uniq_keys = np.array([i+(output_sequence_len*k) for k in range(output_sequence_len)])
                y_temp = y_temp.reshape(2,sets,output_sequence_len)
                arrays_split = np.hsplit(y_temp,sets)
                dict_y.update(dict(zip(uniq_keys, arrays_split)))
              

              temp_x_list  = [dict_x[i].T for i in sorted(dict_x.keys())]
              temp_y_list  = [dict_y[i].T for i in sorted(dict_y.keys())]

              #_country_code,popData2018
              xy_list = [Data(x = torch.from_numpy(features).type(torch.FloatTensor).squeeze()) for features in temp_x_list]

              for i in sorted(dict_y.keys()):
                xy_list[i].y = torch.from_numpy(temp_y_list[i]).squeeze()

              data_list += xy_list
          print('processed : '+ raw_path)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [0]:
class RNNModel(nn.Module):
    def __init__(self, input_size, parameter_sizes, repeats ,output_size):
        super(RNNModel, self).__init__()
        self.input_size = input_size
        self.repeater_input_size = parameter_sizes[0]
        self.hidden_size = parameter_sizes[1]
        self.repeats = repeats
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, self.repeater_input_size, bias=False )
        self.relu_activation = nn.ReLU()

        self.layers = dict()

        k = 0
        for i in range(repeats):
          i = i+k
          self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size, bias=False )
          self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size, bias=False )
          self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size, bias=False )
          k+=2

        self.module_list = nn.ModuleDict(self.layers)

        self.fc2 = nn.Linear(self.repeater_input_size, output_size, bias=False )
        
    def forward(self, input, hidden):
      output = self.fc1(input)
      output = self.relu_activation(output)

      k = 0
      for i in range(self.repeats):
        i = i+k
        output = self.layers['fc_'+str(i)](output)
        output = self.relu_activation(output)

        output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
        output = self.relu_activation(output)
        hidden[i-k] = self.relu_activation(hidden[i-k])

        output = self.layers['fc_'+str(i+2)](output)
        output = self.relu_activation(output)
        k +=2

      output = self.fc2(output)
      output = self.relu_activation(output)

      return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [0]:
def train_rnn(input_tensor, target_tensor, rnn_model, rnn_model_optimizer, criterion,batch_size):
  input_tensor = input_tensor.type(torch.FloatTensor)
  target_tensor= target_tensor.type(torch.FloatTensor)

  rnn_model_hidden = []
  for i in range(rnn_model.repeats):
    rnn_model_hidden.append(rnn_model.initHidden(batch_size))

  rnn_model_optimizer.zero_grad()
  
  loss = 0  
  for ei in range(configuration['input_sequence_len']):
    target_tensor_seq = target_tensor.view(configuration['output_sequence_len'], batch_size, configuration['yhat_size'])
    input_tensor_seq = input_tensor.view(configuration['input_sequence_len'],batch_size, configuration['feature_len'])[ei]
    input_tensor_seq = input_tensor_seq.view(1, batch_size, configuration['feature_len']).to(device)

    rnn_model_output, rnn_model_hidden = rnn_model(
        input_tensor_seq, rnn_model_hidden)
    #print(rnn_model_output.size(), target_tensor_seq[ei].size())
    loss += criterion(rnn_model_output.squeeze(), target_tensor_seq[ei].squeeze().to(device))

  loss.backward()
  rnn_model_optimizer.step()

  return loss.item() / configuration['input_sequence_len']

In [0]:
def evaluate_rnn(input_tensor, target_tensor, rnn_model, criterion,batch_size):
  with torch.no_grad():
    input_tensor = input_tensor.type(torch.FloatTensor)
    target_tensor= target_tensor.type(torch.FloatTensor)

    rnn_model_hidden = []
    for i in range(rnn_model.repeats):
      rnn_model_hidden.append(rnn_model.initHidden(batch_size))

    validation_loss = 0
    
    for ei in range(configuration['input_sequence_len']):
      target_tensor_seq = target_tensor.view(configuration['output_sequence_len'], batch_size, configuration['yhat_size'])
      input_tensor_seq = input_tensor.view(configuration['input_sequence_len'],batch_size, configuration['feature_len'])[ei]
      input_tensor_seq = input_tensor_seq.view(1, batch_size, configuration['feature_len']).to(device)

      rnn_model_output, rnn_model_hidden = rnn_model(
          input_tensor_seq, rnn_model_hidden)
      validation_loss += criterion(rnn_model_output.squeeze(), target_tensor_seq[ei].squeeze().to(device))

  return validation_loss.item() / configuration['input_sequence_len']

In [8]:
configuration = {
    'input_sequence_len' : 10,
    'output_sequence_len' : 10,
    'training_batch_size' : 1024,
    'training_dataset_length' :32768,
    'validation_batch_size' : 1024,
    'yhat_size' : 2,
    'feature_len' : 8,
    'output_size' : 2,
}


INPUT_ROOT = PATH+'/input_mix'
DATA_TAG = "seq2seq_"+str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
PROCESSED_DIR = '/processed_'+DATA_TAG

validation_dataset = CovidDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
validation_dataset = validation_dataset.shuffle()
validation_dataset = validation_dataset[3000:]
validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])

training_dataset = CovidDataSet(INPUT_ROOT+'/training', configuration['input_sequence_len'], configuration['input_sequence_len'])
training_dataset = training_dataset.shuffle()
training_dataset = training_dataset[:configuration['training_dataset_length']]
training_dataloader = DataLoader(training_dataset,batch_size=configuration['training_batch_size'])

#training_dataset = validation_dataset[:3000]
#training_dataset = training_dataset[:configuration['training_dataset_length']]
#training_dataloader = DataLoader(training_dataset,batch_size=configuration['training_batch_size'])

print('batches validation :', len(validation_dataloader), 'training : ', len(training_dataloader))
print('dataset length validation :', len(validation_dataset), 'training : ', len(training_dataset))

batches validation : 7 training :  32
dataset length validation : 6260 training :  32768


In [22]:
def generate_repeat_counts(alphas , training_dataset_length, feature_len,yhat_size,\
                           enc_repeated_nn_parameters = 132096):
  repeats = []
  model1 = RNNModel(**{
         'input_size': configuration['feature_len'],
         'parameter_sizes': [128, 256] ,
         'repeats' : 1 ,
         'output_size' : yhat_size
        })
  model2 = RNNModel(**{
         'input_size': configuration['feature_len'],
         'parameter_sizes': [128, 256] ,
         'repeats' : 2 ,
         'output_size' : yhat_size
        }
  )

  enc_repeated_nn_parameters = get_n_params(model2) - get_n_params(model1)

  for alpha in alphas:
    preferred_parameters  = (1/alpha)* (training_dataset_length / (configuration['feature_len']+configuration['yhat_size']))
    enc_repeats = int(
        (preferred_parameters - 
        (preferred_parameters%enc_repeated_nn_parameters))
        / enc_repeated_nn_parameters)
    for j in range(1, enc_repeats+1):
      if j not in repeats:
        repeats.append(j)
  return repeats

def generate_model_parameters(repeat_counts, feature_len, output_size):
  model_dict = dict()
  model_id = 0
  for enc_repeat in repeat_counts:
    model_dict[model_id] = [{
         'input_size': feature_len,
         'parameter_sizes': [256, 128] ,
         'repeats' : enc_repeat ,
         'output_size' : output_size
        }]
    model_id+=1
  return model_dict

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'repeats' : model_parameters_dict[i][0]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

#Create a new directory to store model results
MODEL_PREFIX = '_rnn'
MODEL_DIR =  PATH + '/models_'+MODEL_PREFIX
dirs = []
for f in listdir(MODEL_DIR):
  if "run" in f:
    dirs.append(int(f[3]))
if os.path.exists(MODEL_DIR+"/run"+str(max(dirs)+1)) == False:
  os.mkdir(MODEL_DIR+"/run"+str(max(dirs)+1))
  MODEL_DIR = MODEL_DIR+"/run"+str(max(dirs)+1)


if os.path.exists(MODEL_DIR)==False:
  os.mkdir(MODEL_DIR)
session_id =str(1001)+ '_'+str(datetime.now(tz=None).microsecond)

repeat_counts = generate_repeat_counts([1,2,3], 1000*len(training_dataset), feature_len = 8, yhat_size=2)

model_configurations = generate_model_parameters(repeat_counts, 8, 2)
save_model_parameters(model_configurations, fileName= MODEL_DIR + '/rnn_model_configurations_'+session_id+'.csv')
models = dict()
print(len(model_configurations))
print(len(repeat_counts))

7
7


In [27]:
#Run all the candidate models
session_id =str(1001)+ '_'+str(datetime.now(tz=None).microsecond)
learning_rate = 0.01
for i in model_configurations:
  #instantiate the models
  if i != 4:
    print('skipping', i)
    continue
  current_model_dir =  MODEL_DIR +'/'+str(i)
  if os.path.exists(current_model_dir) == False:
    os.mkdir(current_model_dir)
  
  #rnn_model = RNNModel(**model_configurations[i][0]).to(device)
  rnn_model = current_model
  rnn_model_optimizer= current_optimizer
  criterion = nn.MSELoss()

  losses = trainValidationIters(training_dataloader,validation_dataloader,
                    rnn_model,
                    rnn_model_optimizer,
                    criterion, n_iters = 500,output_sequence_len=configuration['output_sequence_len'],
                    input_sequence_len=configuration['input_sequence_len'],
                    current_model_dir=current_model_dir,  model_id = i,session_id = session_id,
                     print_every=10, save_every = 50)
  save_dict({
      'model' : rnn_model.state_dict(),
      'rnn_model_optimizer' : rnn_model_optimizer.state_dict(),
      'losses' : losses
  }, current_model_dir+'/'+str(i)+'models_'+session_id+'.pt')

  print('Model ',str(i), ': trained and evaluated')
  current_model = rnn_model
  current_optimizer = rnn_model_optimizer
  models[i] = {
      'rnn_model' : rnn_model.state_dict(),
      'rnn_model_optimizer' : rnn_model_optimizer.state_dict(),
      'losses' : losses
  }

save_dict(models, MODEL_DIR+'/models_'+session_id+'.pt')

skipping 0
skipping 1
skipping 2
skipping 3
1m 28s (- 72m 15s) (10 2%) training:1378.9811 validation:1474.7791
train/validation difference :  95.79790981687393
2m 56s (- 70m 37s) (20 4%) training:1364.3038 validation:1463.1957
train/validation difference :  98.89191102539985
4m 24s (- 69m 7s) (30 6%) training:1364.2940 validation:1463.1241
train/validation difference :  98.83011553834808
5m 52s (- 67m 28s) (40 8%) training:1364.2882 validation:1463.0816
train/validation difference :  98.79332666932669
7m 19s (- 65m 55s) (50 10%) training:1364.2845 validation:1463.0544
train/validation difference :  98.76990123955511
8m 48s (- 64m 37s) (60 12%) training:1364.2821 validation:1463.0364
train/validation difference :  98.75434784424101
10m 16s (- 63m 8s) (70 14%) training:1364.2804 validation:1463.0241
train/validation difference :  98.74368487578863
11m 44s (- 61m 38s) (80 16%) training:1364.2792 validation:1463.0155
train/validation difference :  98.73628655945754
13m 11s (- 60m 5s) (90 1

KeyboardInterrupt: ignored

In [0]:

torch.save(current_model.state_dict(), current_model_dir+'/model_manual_save_2312_5.pt')
torch.save(current_optimizer.state_dict(), current_model_dir+'/optimizer_manual_save_2312_5.pt')

In [0]:
def train_batch(dataloader, rnn_model, rnn_model_optimizer, criterion):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/configuration['input_sequence_len'])
    loss += (train_rnn(batch.x.view(batch_size, configuration['input_sequence_len'] , configuration['feature_len']), batch.y, rnn_model,
                      rnn_model_optimizer, criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def batch_evaluate(dataloader, rnn_model, criterion, output_sequence_len):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/configuration['input_sequence_len'])
    loss += (evaluate_rnn(batch.x.view(batch_size, configuration['input_sequence_len'] , configuration['feature_len']), batch.y, rnn_model,
                     criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def trainValidationIters(train_dataloader,validation_dataloader, rnn_model, rnn_model_optimizer, 
                        criterion ,n_iters, output_sequence_len,input_sequence_len,
                        current_model_dir, model_id,session_id,print_every=1000, plot_every=100,save_every = 100):

  start = time.time()
  plot_losses = []
  
  # Reset every print_every # Reset every plot_every
  print_training_loss_total = 0
  plot_training_loss_total = 0
  print_validation_loss_total = 0
  plot_validation_loss_total = 0
  save_validation_loss_total = 0
  save_training_loss_total = 0
      
  losses = []

  for iter in range(1, n_iters + 1):
      rnn_model.train()
      training_loss = train_batch(train_dataloader, rnn_model,
                                  rnn_model_optimizer, criterion )
      
      rnn_model.eval()
      validation_loss = batch_evaluate(validation_dataloader, rnn_model,
                                       criterion,configuration['output_sequence_len'] )
      
      print_training_loss_total += training_loss
      plot_training_loss_total += training_loss
      print_validation_loss_total += validation_loss
      plot_validation_loss_total += validation_loss
      
      save_training_loss_total += training_loss
      save_validation_loss_total += validation_loss
      
      if iter % print_every == 0:
          print_training_loss_avg = print_training_loss_total / print_every
          print_training_loss_total = 0
          print_validation_loss_avg = print_validation_loss_total / print_every
          print_validation_loss_total = 0
          print('%s (%d %d%%) training:%.4f validation:%.4f' % (timeSince(start, iter / n_iters),
                                        iter, iter / n_iters * 100, print_training_loss_avg,print_validation_loss_avg, ))
          print('train/validation difference : ',print_validation_loss_avg-print_training_loss_avg)

      #if iter % plot_every == 0:
      #    plot_training_loss_avg = plot_training_loss_total / plot_every
      #    plot_validation_loss_avg = plot_validation_loss_total / plot_every
      #    plot_training_loss.append(plot_training_loss_avg)
      #    plot_validation_loss.append(plot_validation_loss_avg)
      #    showPlot(plot_training_losses,plot_validation_loss)
      #    plot_training_loss_total = 0
      #    plot_validation_loss_total
        
      if iter % save_every == 0:
        save_training_loss_avg = save_training_loss_total / save_every
        save_validation_loss_avg = plot_validation_loss_total / save_every
        
        #'arch': args.arch,
        save_checkpoint({
            'epoch': iter + 1,
            'rnn_model_state_dict': rnn_model.state_dict(),
            'training_loss': save_training_loss_avg,
            'validation_loss': save_validation_loss_avg,
            'rnn_model_optimizer' : rnn_model_optimizer.state_dict(),
        }, False, filename=current_model_dir+'/'+str(iter)+'_'+str(session_id)+'.pt')
      losses.append([training_loss, validation_loss])

  save_dict({
      'losses' : losses
  }, filename=current_model_dir+'/losses_'+session_id)

  return losses

In [0]:
"""
Common function
"""
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
  

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'encoder repeats' : model_parameters_dict[i][0]['repeats'],
                        'decoder repeats' : model_parameters_dict[i][1]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

#Generate the list of repeat count parameters list

#generate_model_parameters(generate_repeat_counts())

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def save_dict(dictionary, filename):
  np.save(filename, dictionary)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points, points2):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.plot(points)
    plt.plot(points2)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
